## Simulation in the Inventory Model ( Julia )

This code simulates the inventory of a firm many times.

It generates a matrix with `num_paths` rows and `sim_length` columns.

One simulation of inventory dynamics corresponds to one row.  

The question is: how well does naive parallelization work?

In [1]:
addprocs(3)
@everywhere using Distributions

In [2]:
@everywhere mutable struct SP
    s::Float64
    S::Float64
    p::Float64
end


@everywhere function SP()
    
    s = 10.0
    S = 100.0
    p = 0.4
    SP(s, S, p)   
end

## Sequential:

In [3]:
function sim_paths(sp::SP;initial_x=50.0, num_paths=100000,sim_length=400)
    
    X = Array{Int32}(num_paths,sim_length)
    X[:, 1] = initial_x
    
    # For each row
    for i = 1:num_paths
        
        dvals = rand(Geometric(sp.p),sim_length)
        for t = 1:sim_length-1
            x, d = X[i, t], dvals[t]
            if x <= sp.s
                y = max(sp.S - d, 0)
            else
                y = max(x - d, 0)
            end
            
            X[i, t+1] = y
            
            end
        end
       return X
    
end

sim_paths (generic function with 1 method)

In [4]:
sp = SP();

compilation time:

In [5]:
@time X = sim_paths(sp);

  2.418546 seconds (40.49 M allocations: 1.062 GiB, 9.73% gc time)


running time:

In [6]:
@time X = sim_paths(sp)

  2.256507 seconds (40.46 M allocations: 1.060 GiB, 7.80% gc time)


100000×400 Array{Int32,2}:
 50  50  45  41  39  39  32  32  29  26  …  53  52  52  52  52   52   49  49
 50  50  50  47  47  47  37  36  35  34     40  38  36  36  35   35   34  31
 50  50  50  44  42  42  38  38  38  38     35  35  35  31  31   29   28  26
 50  49  49  48  48  47  47  46  44  44     41  40  39  35  34   34   29  28
 50  49  44  43  42  40  37  35  33  30     50  49  49  48  48   48   44  42
 50  49  40  39  38  38  33  30  28  28  …  19  18  14  13  10  100  100  99
 50  47  47  47  45  44  44  44  43  43     81  81  78  77  76   76   76  75
 50  49  49  46  45  43  43  43  38  37     42  40  40  38  37   37   36  36
 50  49  45  45  45  45  44  42  42  41     75  71  71  68  67   63   61  61
 50  48  48  48  47  43  38  38  36  36     81  81  81  79  78   78   78  78
 50  50  50  48  48  46  40  39  38  36  …  34  34  32  31  31   30   25  24
 50  39  39  38  38  37  35  33  31  31     46  46  46  46  44   42   40  38
 50  48  45  45  45  45  41  40  37  34     87  8

## Parallel:

In [7]:
@everywhere function sim_paths(sp::SP;initial_x=50.0, 
                                num_paths=100000, sim_length=400)
    
    X = SharedArray{Int32}(num_paths,sim_length)
    X[:, 1] = initial_x
    
    # For each row
    @sync @parallel for i=1:num_paths
        dvals = rand(Geometric(sp.p),sim_length)
        
        for t=1:sim_length-1
            x, d = X[i, t], dvals[t]
            
            if x <= sp.s
                y = max(sp.S - d, 0)
            else
                y = max(x - d, 0)
            end
        
            X[i, t+1] = y
            
        end
    end
    
    return X  
end

compilation time:

In [8]:
@time X = sim_paths(sp);

  3.585349 seconds (1.44 M allocations: 80.589 MiB, 0.92% gc time)


running time:

In [9]:
@time X = sim_paths(sp)

  0.753148 seconds (2.50 k allocations: 129.229 KiB)


100000×400 SharedArray{Int32,2}:
 50  50  43  43  43  43  42  42  39  39  …   12   11   9   99   99   97   95
 50  49  48  48  46  46  46  46  46  44      87   84  84   81   81   81   79
 50  50  49  39  38  36  36  36  36  32     100   98  97   96   95   95   95
 50  50  46  46  43  43  42  41  40  40      21   18  17   14   14   13   13
 50  49  47  46  45  41  41  41  39  38      48   48  46   46   46   45   45
 50  50  50  50  50  48  46  46  43  43  …   64   64  64   64   64   61   61
 50  44  44  43  42  39  39  39  39  35      42   42  42   42   36   36   36
 50  49  44  42  42  38  37  37  37  37      94   93  92   91   90   90   90
 50  49  47  46  38  38  37  35  35  31      40   37  34   34   28   27   24
 50  50  46  44  44  42  42  40  39  38       6  100  98   98   97   96   96
 50  46  45  43  39  37  37  36  36  33  …   71   71  69   69   68   64   63
 50  50  50  49  48  48  47  47  44  43      30   30  30   30   27   26   26
 50  50  48  48  45  42  42  40  40  37    